In [5]:
!pip install aif360
!pip install BlackBoxAuditing
!pip install fairlearn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.7/259.7 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 43.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for BlackBoxAuditing: filename=BlackBoxAuditing-0.1.54-py2.py3-none-any.whl size=1394756 sha256=75aee4968959ac6e44557b8bf7f95220b71f0c89c49734104a35ffa05d9bbec3
  Stored in directory: /root/.cache/pip/wheels/c9/8c/03/073e80e604151fb4cdc68b2e56a97f338d7723e4a4ab5e3823
Successfully built BlackBoxAuditing
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.0/240.0 kB 5.6 MB/s eta 0:00:00


In [6]:
import pandas as pd
import numpy as np

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from aif360.datasets import StandardDataset
from aif360.algorithms.preprocessing import Reweighing
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.algorithms.postprocessing import EqOddsPostprocessing

pip install 'aif360[inFairness]'


In [18]:

# Set random seed for reproducibility
np.random.seed(42)

# Generate 100 entries (50 Male, 50 Female)
num_samples = 100
genders = ['Male'] * 50 + ['Female'] * 50
np.random.shuffle(genders)  # Shuffle gender order

# Age: Different distributions by gender (males slightly older on average)
ages = []
for gender in genders:
    if gender == 'Male':
        ages.append(int(np.random.normal(loc=38, scale=8)))
    else:
        ages.append(int(np.random.normal(loc=35, scale=7)))
ages = np.clip(ages, 22, 60)  # Cap ages between 22-60

# Experience: Correlated with age but more varied for males
experiences = []
for age, gender in zip(ages, genders):
    base_exp = age - 22  # Minimum hiring age
    if gender == 'Male':
        experiences.append(int(base_exp + np.random.normal(scale=3)))
    else:
        experiences.append(int(base_exp + np.random.normal(scale=2)))
experiences = np.clip(experiences, 0, 40)  # Cap experience

# Hiring decision: Biased against females
hired = []
for age, gender, exp in zip(ages, genders, experiences):
    # Base probability increases with experience
    prob = 0.3 + 0.5 * (exp / 20)

    # Strong bias against females
    if gender == 'Female':
        prob *= 0.5  # 50% less likely to be hired

    # Final hiring decision with some randomness
    hired.append(int(np.random.random() < prob))

# Create DataFrame
data = pd.DataFrame({
    'Age': ages,
    'Gender': genders,
    'Experience': experiences,
    'Hired': hired
})

# Verify distributions
print("Hiring rates by gender:")
print(data.groupby('Gender')['Hired'].mean())

print("\nAverage experience by gender:")
print(data.groupby('Gender')['Experience'].mean())

print("\nFirst 100 entries:")
print(data.head(100))

Hiring rates by gender:
Gender
Female    0.36
Male      0.70
Name: Hired, dtype: float64

Average experience by gender:
Gender
Female    12.24
Male      15.64
Name: Experience, dtype: float64

First 100 entries:
    Age  Gender  Experience  Hired
0    39  Female          13      0
1    28  Female           6      0
2    38  Female          18      1
3    48    Male          26      1
4    39    Male          15      1
..  ...     ...         ...    ...
95   41  Female          17      0
96   36  Female          13      0
97   26    Male           1      1
98   27  Female           4      1
99   40  Female          18      0

[100 rows x 4 columns]


In [19]:

# Convert categorical Gender column to binary
data['Gender'] = data['Gender'].map({'Male': 1, 'Female': 0})

# Define privileged and unprivileged groups
privileged_groups = [{'Gender': 1}]
unprivileged_groups = [{'Gender': 0}]


In [21]:

# Convert to AIF360 dataset
aif_data = StandardDataset(
    df=data,
    label_name='Hired',
    favorable_classes=[1],
    protected_attribute_names=['Gender'],  # Removed 'Age' as it's not used for fairness
    privileged_classes=[[1]],  # Only Gender is used for fairness
    instance_weights_name=None
)


In [22]:

# Analyze bias before mitigation
metric = BinaryLabelDatasetMetric(aif_data,
                                unprivileged_groups=unprivileged_groups,
                                privileged_groups=privileged_groups)
print(f"Statistical Parity Difference (Before Mitigation): {metric.statistical_parity_difference()}")
print(f"Disparate Impact (Before Mitigation): {metric.disparate_impact()}")


Statistical Parity Difference (Before Mitigation): -0.33999999999999997
Disparate Impact (Before Mitigation): 0.5142857142857143


In [23]:

# Apply Reweighing to mitigate bias
rw = Reweighing(privileged_groups=privileged_groups,
               unprivileged_groups=unprivileged_groups)
reweighted_data = rw.fit_transform(aif_data)

# Split data into training and test sets (before scaling)
train, test = reweighted_data.split([0.8], shuffle=True, seed=42)


In [24]:

# Standardize features
scaler = StandardScaler()
X_train = train.features
y_train = train.labels.ravel()
X_test = test.features
y_test = test.labels.ravel()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [25]:

# Train Logistic Regression model
model = LogisticRegression()
model.fit(X_train_scaled, y_train)

# Evaluate model
preds = model.predict(X_test_scaled)
print("\nModel Accuracy After Bias Mitigation:", accuracy_score(y_test, preds))
print("\nClassification Report:")
print(classification_report(y_test, preds))



Model Accuracy After Bias Mitigation: 0.55

Classification Report:
              precision    recall  f1-score   support

         0.0       0.36      0.67      0.47         6
         1.0       0.78      0.50      0.61        14

    accuracy                           0.55        20
   macro avg       0.57      0.58      0.54        20
weighted avg       0.65      0.55      0.57        20



In [26]:

# Create a copy of the test dataset for fairness evaluation
aif_test = test.copy()
aif_test.labels = preds.reshape(-1, 1)

# Evaluate fairness after mitigation
metric_after = BinaryLabelDatasetMetric(aif_test,
                                      unprivileged_groups=unprivileged_groups,
                                      privileged_groups=privileged_groups)
print(f"\nStatistical Parity Difference (After Mitigation): {metric_after.statistical_parity_difference()}")
print(f"Disparate Impact (After Mitigation): {metric_after.disparate_impact()}")


Statistical Parity Difference (After Mitigation): -0.557826495567313
Disparate Impact (After Mitigation): 0.2299994359706068
